### Imports section

In [1]:
import pandas as pd
import numpy as numpy
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [6]:
df_shops = pd.read_csv('shops.csv')
df = pd.read_csv('sales_train.csv')
df.shape

(2935849, 6)

### Price and item count cleaning (basically on main df)

In [9]:
df = df[df.item_price < 100000]
df = df[df.item_cnt_day < 1001]

# replace negative price with median price of same shop-item-month combination
median = df[(df.shop_id == 32) & (df.item_id == 2973) & (df.date_block_num == 4) & (
            df.item_price > 0)].item_price.median()
df.loc[df.item_price < 0, 'item_price'] = median

#### Replacing shop_id for same shops

In [10]:
# Yakutsk Ordzhonikidze, 56  
df.loc[df.shop_id == 0, 'shop_id'] = 57

# Yakutsk TC "Central" franc 
df.loc[df.shop_id == 1, 'shop_id'] = 58

# Zhukovsky Str. Chkalov 39m?   
df.loc[df.shop_id == 10, 'shop_id'] = 11


### Function for shop (pass to it the base shop df) return df with city and location

In [ ]:
def get_shop_df(shops):
    #Fix error with the name
    shops.loc[ shops.shop_name == 'Сергиев Посад ТЦ "7Я"',"shop_name" ] = 'СергиевПосад ТЦ "7Я"'
    shops["city"] = shops.shop_name.str.split(" ").map( lambda x: x[0] )
    shops["location"] = shops.shop_name.str.split(" ").map( lambda x: x[1] )
    #Clean this dirty city
    shops.loc[shops.city == "!Якутск", "city"] = "Якутск"
    
    #Store in cat_vec the most frequent position of shops, if position is present less than 4 time
    cat_vec = []
    for cat in shops.location.unique():
        if len(shops[shops.location == cat]) >= 4:
            cat_vec.append(cat)
    #put it on external
    shops.location = shops.location.apply( lambda x: x if (x in cat_vec) else "external" )
    return shops

### df_shop(modified)

In [ ]:
df_shops = get_shop_df(df_shops)
df_shops.sample(5)

,shop_name,shop_id,city,location
31,"Москва ТЦ ""Семеновский""",31,Москва,ТЦ
52,"Уфа ТК ""Центральный""",52,Уфа,ТК
5,"Вологда ТРЦ ""Мармелад""",5,Вологда,ТРЦ
9,Выездная Торговля,9,Выездная,external
32,"Москва ТЦ ""Серебряный Дом""",32,Москва,ТЦ


### Functions for scores

In [ ]:
def item_score(df):
    item_score = df.groupby(['item_id']).item_cnt_day.sum().reset_index().sort_values(by='item_cnt_day', ascending=False)
    item_score['item_score'] = item_score['item_cnt_day'] / item_score.item_cnt_day.sum()
    item_score.drop(columns='item_cnt_day', inplace=True)
    df_item_score = pd.merge(df, item_score, how='left', on=["item_id"])
    return df_item_score

In [ ]:
def shop_score(df):
    shop_score = df.groupby(['shop_id']).item_cnt_day.sum().reset_index().sort_values(by='item_cnt_day', ascending=False)
    shop_score['shop_score'] = shop_score['item_cnt_day'] / shop_score.item_cnt_day.sum()
    shop_score.drop(columns='item_cnt_day', inplace=True)
    df_shop_score = pd.merge(df, shop_score, how='left', on=["shop_id"])
    return df_shop_score

In [ ]:
df = item_score(df)

In [ ]:
df = shop_score(df)

In [ ]:
df

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_score,shop_score
0,02.01.2013,0,59,22154,999.00,1.0,1.618196e-05,0.013437
1,03.01.2013,0,25,2552,899.00,1.0,1.371352e-06,0.066352
2,05.01.2013,0,25,2552,899.00,-1.0,1.371352e-06,0.066352
3,06.01.2013,0,25,2554,1709.05,1.0,2.742705e-07,0.066352
4,15.01.2013,0,25,2555,1099.00,1.0,1.535915e-05,0.066352
...,...,...,...,...,...,...,...,...
2935842,10.10.2015,33,25,7409,299.00,1.0,4.662598e-06,0.066352
2935843,09.10.2015,33,25,7460,299.00,1.0,3.181537e-05,0.066352
2935844,14.10.2015,33,25,7459,349.00,1.0,1.560599e-04,0.066352
2935845,22.10.2015,33,25,7440,299.00,1.0,6.308221e-06,0.066352


In [ ]:

df1 = df[df['date_block_num'] == 10]
df2 = df[df['date_block_num'] == 22]
df3 = df1.append(df2)
df4 = df3[['shop_id','item_id', 'item_price']]
data_grouping = df1.append(df2)
grouped = data_grouping.groupby(['date_block_num', 'shop_id', 'item_id'])['item_cnt_day'].sum().reset_index()
test = pd.read_csv('test.csv', index_col='ID')
merged = pd.merge(test, df4, how='left',  on=['shop_id', 'item_id'])
train_merged = pd.merge(grouped, df4, how='left', on=['shop_id', 'item_id'])
merged = merged.dropna()
merged['date_block_num'] = 34
df5 = df4
df5['date_block_num'] = df3['date_block_num']
df6 = train_merged.append(merged)
#score_df = df[['shop_id', 'item_id', 'item_score', 'shop_score']]
#final_merge = pd.merge(df6, score_df, how='left', on=['shop_id', 'item_id'])
df6
final_merge = pd.merge(df6, df_shops, how='left', on=['shop_id'])
#final_merge = final_merge.drop('shop_name', axis=1)
#final_merge_cat = final_merge[['date_block_num', 'city', 'location']]

enc = OneHotEncoder()
enc.fit(final_merge_cat)
final_merge_cat = enc.transform(final_merge_cat)
final_merge_cat


final_merge

# give test a column of date that has 34 in it
# append test to df
# get cat
# ohe them
# scale num
# concat encoded cat with encoded num
# train test split
# train model
# get result
# merge result with test 
# fill nans with 0s

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,date_block_num,shop_id,item_id,item_cnt_day,item_price,shop_name,city,location
0,10,2,33,2.0,199.0,"Адыгея ТЦ ""Мега""",Адыгея,ТЦ
1,10,2,33,2.0,199.0,"Адыгея ТЦ ""Мега""",Адыгея,ТЦ
2,10,2,97,1.0,149.0,"Адыгея ТЦ ""Мега""",Адыгея,ТЦ
3,10,2,482,2.0,3300.0,"Адыгея ТЦ ""Мега""",Адыгея,ТЦ
4,10,2,482,2.0,3300.0,"Адыгея ТЦ ""Мега""",Адыгея,ТЦ
...,...,...,...,...,...,...,...,...
305415,34,45,3316,NaN,199.0,"Самара ТЦ ""ПаркХаус""",Самара,ТЦ
305416,34,45,7543,NaN,149.0,"Самара ТЦ ""ПаркХаус""",Самара,ТЦ
305417,34,45,19889,NaN,149.0,"Самара ТЦ ""ПаркХаус""",Самара,ТЦ
305418,34,45,15757,NaN,199.0,"Самара ТЦ ""ПаркХаус""",Самара,ТЦ


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f264f3ab-4d0e-489a-8f83-6b7eca180fc5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>